# Train Normal Model

In [1]:
from cv2 import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import os.path

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models

from pprint import pprint as pp

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2021-11-23 01:17:17.423180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:17:17.428925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:17:17.429322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
from asl_model import get_model_3

TRAIN_DIR = 'DATASET_A_AP/train'
TEST_DIR = 'DATASET_A_AP/test'

BATCH = 128
EPOCH = 100

model_func = get_model_3

checkpoint_path = "ckpt_app_training/cp.ckpt"
tb_log_dir = 'app-logs/'
entire_model_s_path = 'saved_model/app_model'

In [4]:
# Convert folder to dataframe of images' paths & labels
def get_paths_labels(path, allowed_extension="jpg"):
        global Path
        images_dir = Path(path)
        
        filepaths = pd.Series((images_dir.glob(fr'**/*.{allowed_extension}'))).astype(str)
        filepaths.name = "path"
        
        labels = filepaths.str.split("/")[:].str[-2]
        labels.name = "label"

        # Concatenate filepaths and labels
        df = pd.concat([filepaths, labels], axis=1)

        # Shuffle the DataFrame and reset index
        df = df.sample(frac=1).reset_index(drop = True)
        return df

In [5]:
train_df = get_paths_labels(TRAIN_DIR)
test_df = get_paths_labels(TEST_DIR)

In [6]:
data_generator = ImageDataGenerator(validation_split=0.2,
                                    rescale=1. / 255.,
                                    zoom_range=0.1,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.1,)

test_generator = ImageDataGenerator(rescale=1. / 255.)

train_images = data_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH,
    shuffle=True,
    subset='training'
)

val_images = data_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH,
    shuffle=True,
    subset='validation'
)


test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    # batch_size=1,
    shuffle=False,
)

Found 46174 validated image filenames belonging to 26 classes.
Found 11543 validated image filenames belonging to 26 classes.
Found 14430 validated image filenames belonging to 26 classes.


# Callback Settings

In [7]:
# checkpoint_path = "ckpt_normal_training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=2, 
    save_weights_only=True,
    # save_best_only=True,
    save_freq=5*BATCH
)


es_callback = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau()


tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)

callback_ls = [
    es_callback, 
    reduce_lr_callback, 
    cp_callback,
    tb_callback,
]

In [8]:
from asl_model import *

def create_model():
    model = get_model_3()
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [9]:
model = create_model()

model.fit(train_images, 
          validation_data=val_images,
          epochs=EPOCH,
          callbacks=callback_ls)

2021-11-23 01:17:18.965648: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-23 01:17:18.966474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:17:18.966870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:17:18.967207: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/100


2021-11-23 01:17:20.776240: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204


361/361 [==============================] - ETA: 0s - loss: 1.8420 - accuracy: 0.5919

2021-11-23 01:17:46.064066: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7612661760 exceeds 10% of free system memory.


...

361/361 [==============================] - 25s 69ms/step - loss: 0.3850 - accuracy: 0.8947 - val_loss: 0.1915 - val_accuracy: 0.9540 - lr: 0.0010
Epoch 4/100
196/361 [===============>..............] - ETA: 8s - loss: 0.2798 - accuracy: 0.9203
Epoch 00004: saving model to ckpt_app_training/cp.ckpt
360/361 [============================>.] - ETA: 0s - loss: 0.2679 - accuracy: 0.9236

2021-11-23 01:19:02.248268: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7612661760 exceeds 10% of free system memory.


361/361 [==============================] - 26s 71ms/step - loss: 0.2683 - accuracy: 0.9235 - val_loss: 0.1356 - val_accuracy: 0.9666 - lr: 0.0010
Epoch 5/100
361/361 [==============================] - ETA: 0s - loss: 0.2029 - accuracy: 0.9436

2021-11-23 01:19:27.435929: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7612661760 exceeds 10% of free system memory.


361/361 [==============================] - 25s 70ms/step - loss: 0.2029 - accuracy: 0.9436 - val_loss: 0.1153 - val_accuracy: 0.9695 - lr: 0.0010
Epoch 6/100
114/361 [========>.....................] - ETA: 12s - loss: 0.2025 - accuracy: 0.9452
Epoch 00006: saving model to ckpt_app_training/cp.ckpt
361/361 [==============================] - 26s 71ms/step - loss: 0.1934 - accuracy: 0.9475 - val_loss: 0.1396 - val_accuracy: 0.9624 - lr: 0.0010
Epoch 7/100
361/361 [==============================] - 25s 70ms/step - loss: 0.1568 - accuracy: 0.9568 - val_loss: 0.0825 - val_accuracy: 0.9755 - lr: 0.0010
Epoch 8/100
 32/361 [=>............................] - ETA: 16s - loss: 0.1415 - accuracy: 0.9634
Epoch 00008: saving model to ckpt_app_training/cp.ckpt
361/361 [==============================] - 25s 71ms/step - loss: 0.1439 - accuracy: 0.9598 - val_loss: 0.0921 - val_accuracy: 0.9746 - lr: 0.0010
Epoch 9/100
311/361 [========================>.....] - ETA: 2s - loss: 0.1331 - accuracy: 0.9630
E

# save entire model

In [10]:
model.save(entire_model_s_path)

2021-11-23 01:30:45.996053: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/app_model/assets


# reload the entire model

In [11]:
new_model = tf.keras.models.load_model(entire_model_s_path)

In [12]:
new_model.evaluate(test_images)

451/451 [==============================] - 5s 10ms/step - loss: 0.4442 - accuracy: 0.9341


[0.44423216581344604, 0.9340956211090088]

# reload weight by checkpoint

In [13]:
new_model = create_model()

In [14]:
new_model.load_weights(checkpoint_path)

In [15]:
loss, acc = new_model.evaluate(test_images)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

451/451 [==============================] - 4s 9ms/step - loss: 0.4039 - accuracy: 0.9204
Restored model, accuracy: 92.04%
